In [1]:
# Import numpy and OpenCV
import numpy as np
import cv2

# Read input video
cap = cv2.VideoCapture('WhatsApp Video 2022-05-16 at 4.20.24 PM (1).mp4')

# Get frame count
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fps = int(cap.get(cv2.CAP_PROP_FPS))

# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec for output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# Set up output video
print(fps)
#out = cv2.VideoWriter('video_out.mp4', fourcc, fps, (w, h))
print(w)
print(h)

29
352
640


In [2]:
# Read first frame
_, prev = cap.read() 

# Convert frame to grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

In [3]:
def findAffine(src, dst, fullAffine=False):
    #print("src:", src)
    #print("dst:", dst)
    if len(src) >= affine_minpts:
        # affine = cv2.estimateRigidTransform(np.array([src]), np.array([dst]), fullAffine)
        affine, status = \
            cv2.estimateAffinePartial2D(np.array([src]).astype(np.float32),
                                        np.array([dst]).astype(np.float32))
    else:
        affine = None
    #print str(affine)
    return affine 

In [4]:
# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32) 

for i in range(n_frames-2):
  # Detect feature points in previous frame
  prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)

  # Read next frame
  success, curr = cap.read()
  if not success:
    break 

  # Convert to grayscale
  curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY) 

  # Calculate optical flow (i.e. track feature points)
  curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 

  # Sanity check
  assert prev_pts.shape == curr_pts.shape 

  # Filter only valid points
  idx = np.where(status==1)[0]
  prev_pts = prev_pts[idx]
  curr_pts = curr_pts[idx]

  #Find transformation matrix
  m = cv2.estimateAffinePartial2D(prev_pts, curr_pts)[0]

  # Extract traslation
  dx = m[0,2]
  dy = m[1,2]

  # Extract rotation angle
  da = np.arctan2(m[1,0], m[0,0])

  # Store transformation
  transforms[i] = [dx,dy,da]

  # Move to next frame
  prev_gray = curr_gray

  print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

Frame: 0/447 -  Tracked points : 80
Frame: 1/447 -  Tracked points : 83
Frame: 2/447 -  Tracked points : 78
Frame: 3/447 -  Tracked points : 74
Frame: 4/447 -  Tracked points : 75
Frame: 5/447 -  Tracked points : 75
Frame: 6/447 -  Tracked points : 72
Frame: 7/447 -  Tracked points : 72
Frame: 8/447 -  Tracked points : 72
Frame: 9/447 -  Tracked points : 72
Frame: 10/447 -  Tracked points : 75
Frame: 11/447 -  Tracked points : 76
Frame: 12/447 -  Tracked points : 75
Frame: 13/447 -  Tracked points : 78
Frame: 14/447 -  Tracked points : 77
Frame: 15/447 -  Tracked points : 77
Frame: 16/447 -  Tracked points : 72
Frame: 17/447 -  Tracked points : 74
Frame: 18/447 -  Tracked points : 80
Frame: 19/447 -  Tracked points : 77
Frame: 20/447 -  Tracked points : 73
Frame: 21/447 -  Tracked points : 76
Frame: 22/447 -  Tracked points : 78
Frame: 23/447 -  Tracked points : 76
Frame: 24/447 -  Tracked points : 76
Frame: 25/447 -  Tracked points : 77
Frame: 26/447 -  Tracked points : 73
Frame: 27/4

Frame: 228/447 -  Tracked points : 48
Frame: 229/447 -  Tracked points : 44
Frame: 230/447 -  Tracked points : 43
Frame: 231/447 -  Tracked points : 46
Frame: 232/447 -  Tracked points : 50
Frame: 233/447 -  Tracked points : 48
Frame: 234/447 -  Tracked points : 57
Frame: 235/447 -  Tracked points : 62
Frame: 236/447 -  Tracked points : 55
Frame: 237/447 -  Tracked points : 43
Frame: 238/447 -  Tracked points : 54
Frame: 239/447 -  Tracked points : 63
Frame: 240/447 -  Tracked points : 61
Frame: 241/447 -  Tracked points : 70
Frame: 242/447 -  Tracked points : 69
Frame: 243/447 -  Tracked points : 67
Frame: 244/447 -  Tracked points : 73
Frame: 245/447 -  Tracked points : 74
Frame: 246/447 -  Tracked points : 72
Frame: 247/447 -  Tracked points : 67
Frame: 248/447 -  Tracked points : 73
Frame: 249/447 -  Tracked points : 74
Frame: 250/447 -  Tracked points : 75
Frame: 251/447 -  Tracked points : 73
Frame: 252/447 -  Tracked points : 71
Frame: 253/447 -  Tracked points : 78
Frame: 254/4

In [5]:
def movingAverage(curve, radius):
  window_size = 2 * radius + 1
  # Define the filter
  f = np.ones(window_size)/window_size
  # Add padding to the boundaries
  curve_pad = np.lib.pad(curve, (radius, radius), 'edge')
  # Apply convolution
  curve_smoothed = np.convolve(curve_pad, f, mode='same')
  # Remove padding
  curve_smoothed = curve_smoothed[radius:-radius]
  # return smoothed curve
  return curve_smoothed

In [6]:
def smooth(trajectory):
  smoothed_trajectory = np.copy(trajectory)
  # Filter the x, y and angle curves
  SMOOTHING_RADIUS=10  
  for i in range(3):
    smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)

  return smoothed_trajectory

In [7]:
# Compute trajectory using cumulative sum of transformations
trajectory = np.cumsum(transforms, axis=0)

In [8]:
smoothed_trajectory=smooth(trajectory)

In [9]:
# Calculate difference in smoothed_trajectory and trajectory
difference = smoothed_trajectory - trajectory

# Calculate newer transformation array
transforms_smooth = transforms + difference

In [10]:
def fixBorder(frame):
  s = frame.shape
  # Scale the image 4% without moving the center
  T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
  frame = cv2.warpAffine(frame, T, (s[1], s[0]))
  return frame

In [23]:
flag=True
while(flag==True):
    # Reset stream to first frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 

    # Write n_frames-1 transformed frames
    for i in range(n_frames-2):
      # Read next frame
      success, frame = cap.read()
      if not success:
        break

      # Extract transformations from the new transformation array
      dx = transforms_smooth[i,0]
      dy = transforms_smooth[i,1]
      da = transforms_smooth[i,2]

      # Reconstruct transformation matrix accordingly to new values
      m = np.zeros((2,3), np.float32)
      m[0,0] = np.cos(da)
      m[0,1] = -np.sin(da)
      m[1,0] = np.sin(da)
      m[1,1] = np.cos(da)
      m[0,2] = dx
      m[1,2] = dy

      # Apply affine wrapping to the given frame
      frame_stabilized = cv2.warpAffine(frame, m, (w,h))

      # Fix border artifacts
      frame_stabilized = fixBorder(frame_stabilized) 

      # Write the frame to the file
      frame_out = cv2.hconcat([frame, frame_stabilized])

      # If the image is too big, resize it.
      if(frame_out.shape[1]> 1920):
        frame_out = cv2.resize(frame_out, (frame_out.shape[1]/2, frame_out.shape[0]/2));
      fourcc = cv2.VideoWriter_fourcc(*'MP4V')
      out = cv2.VideoWriter('output.mp4', fourcc, fps, (352,640))

      cv2.imshow("Before and After", frame_out)
      c=cv2.waitKey(10)
      out.write(frame_out)
      if c & 0xFF == ord('q'):
            out.release()
            cv2.destroyAllWindows()
            flag=False
            break     
    out.release()
    cv2.destroyAllWindows()
    flag=False
